In [ ]:
##se instalan las librerias necesarias
%%time
!pip install datasets==2.16.1
from datasets import Dataset
!pip install huggingface_hub==0.20.3
import os
!pip install langchain==0.1.0
!pip install accelerate==0.25.0
!pip install bitsandbytes==0.42.0
!pip install "unstructured[pdf]"==0.11.8
!pip install sentence-transformers==2.2.2
!pip install faiss-gpu==1.7.2
import torch
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredFileLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
!pip install gradio==4.14.0
import gradio as gr

#se carga el modelo de embeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',model_kwargs={'device': 'cuda'})

##se definen la configuracion de quantization del LLM para alojarlo en la GPU local
bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

#se importa desde huggingface el LLM y se le aplica la configuracion


tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf")


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-13b-chat-hf",
                                             quantization_config=bnb_config,
                                             device_map="auto",
                                             )

##se define el pipeline para generacion de texto

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 1024, ##AQUI SE PUEDE REDUCIR EL NUMERO DE CARACTERES GENERADOS
                do_sample=True,
                top_k=10,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

#se define la temperatura del LLM

llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0}) #CERO PARA REDUCIR LA CREATIVIDAD DEL MODELO

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

CPU times: user 52.2 s, sys: 57.3 s, total: 1min 49s
Wall time: 6min 14s


In [ ]:
def cargar_procesar_configurar_historias_y_responder_preguntas(archivo_xls, pdf_path, pregunta1, pregunta2, pregunta3, pregunta4, pregunta5):
    # Cargar datos desde el archivo Excel
    df = pd.read_excel(archivo_xls, sheet_name='Sheet1')
    df['DATOS'] = df['DATOS'].astype(str).apply(openpyxl.utils.escape.unescape)
    df['DATOS'] = df['DATOS'].str.replace('\r\n', '')
    df['DATOS'] = df['DATOS'].str.replace('\n', '')
    df1 = df[['ID','DATOS']]
    dataset = Dataset.from_pandas(df1)

    # Cargar el PDF y dividirlo en chunks
    loader = UnstructuredFileLoader(pdf_path)
    documents = loader.load()

    text_splitter = CharacterTextSplitter(separator='\n', chunk_size=500, chunk_overlap=50)
    text_chunks = text_splitter.split_documents(documents)

    # Crear la base de datos vectorial ejecutando el modelo de embeddings sobre los chunks
    vectorstore = FAISS.from_documents(text_chunks, embeddings)

    # Establecer el prompt template para ajustar las respuestas recibidas
    prompt_template = """Tu Rol es el de un asistente de auditoría médica de historias clínicas que ayuda a medir la adherencia de una historia clinica a una guía de práctica clínica. Usa las siguientes piezas de context para responder a las preguntas que se presentaran usando solamente SI o NO. No es necesario ampliar el porqué de la respuesta. No es necesario dar el motivo de la respuesta. NO es necesario dar un resumen de la historia. No es necesario dar un resumen de la respuesta.
    Solamente estás autorizado para responder usando SI o NO. No puedes usar ningún otro tipo de texto para responder. No es necesario agregar nada más que SI o NO. No es necesario usar signos de puntuación como . , ; o similares. No te apresures a dar una respuesta, revísala antes de generarla y confirma que cumple con los requisitos especificados.
    Una buena respuesta es aquella que inicia con SI en caso que cumpla o con NO en caso que no cumpla.
    {context}

    Question: {question}
    Respuesta:
"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    # Configurar QA con el prompt template
    chain_type_kwargs = {"prompt": PROMPT}
    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(), chain_type_kwargs=chain_type_kwargs)

    # Crear un nuevo DataFrame para almacenar las respuestas
    df_respuestas = pd.DataFrame(columns=['ID', 'Respuesta1', 'Respuesta2', 'Respuesta3', 'Respuesta4', 'Respuesta5'])

    # Iterar a través de los índices del dataset
    for index in range(len(dataset)):
        # Obtener la historia clínica de la fila actual
        historia_clinica = dataset['DATOS'][index]

        # Hacer la pregunta al sistema y obtener la respuesta
        respuestas = []
        for pregunta in [pregunta1, pregunta2, pregunta3, pregunta4, pregunta5]:
            respuesta = qa.run(f"En {historia_clinica} {pregunta}")
            respuestas.append(respuesta)

        # Agregar la respuesta al nuevo DataFrame
        df_respuestas = df_respuestas.append({
            'ID': dataset['ID'][index],
            'Respuesta1': respuestas[0],
            'Respuesta2': respuestas[1],
            'Respuesta3': respuestas[2],
            'Respuesta4': respuestas[3],
            'Respuesta5': respuestas[4]
        }, ignore_index=True)

    return df_respuestas

def minuscula(df_respuestas):
    # cargar el DF
    df_respuestas = df_respuestas

    return df_respuestas


def limpiar_dataframe(df_respuestas):
    # Función para obtener los primeros dos caracteres de una cadena
    get_first_two_chars = lambda x: x[:2] if isinstance(x, str) else x
    df_limpio = df_respuestas.apply(lambda col: col.apply(get_first_two_chars) if col.name != 'ID' else col)
    df_limpio.loc['Promedio'] = (df_limpio.iloc[:, 1:] == 'SI').mean() *100
    adherencia = df_limpio.loc[['Promedio']]
    adherencia = adherencia.drop(['ID'], axis=1)
    adherencia['Total'] =  adherencia.mean(axis=1)
    return adherencia


def GRAFICO_BARRAS(adherencia):
    fig, ax1 = plt.subplots(figsize=(10, 6))
    adherencia.T.plot(kind='bar', ax=ax1, legend=False)
    ax1.set_ylabel('Porcentaje de Adherencia')
    ax1.set_title('Adherencia por Respuesta')

    # Agregar etiquetas de datos encima de cada columna
    for p in ax1.patches:
        height = p.get_height()
        ax1.annotate(f'{height:.2f}%', (p.get_x() + p.get_width() / 2., height),
                    ha='center', va='bottom', xytext=(0, 5), textcoords='offset points')


    return fig




with gr.Blocks() as demo:
    with gr.Tab("PROCESAR EHR Y GPC"):
      gr.HTML("<p style='text-align:center'><b>INGRESE AQUÍ LA BASE DE DATOS CON HISTORIAS CLÍNICAS EN FORMATO XLS Y LA GPC EN FORMATO PDF</b></p>")
      with gr.Accordion("INGRESAR BDD Y GPC", open=False):
          with gr.Row():
              inp_xls = gr.File(label="Archivo XLS")
              inp_pdf = gr.File(label="Archivo PDF")
      with gr.Accordion("REVISAR PREGUNTAS A EVALUAR", open=False):
          with gr.Row():
              inp_pregunta1 = gr.Textbox(label="Pregunta 1", value='La descripción del examen físico de la próstata, ¿es adecuada según lo especificado en la guía? SI o NO.', lines = 4)
              inp_pregunta2 = gr.Textbox(label="Pregunta 2", value='El tratamiento propuesto para el paciente ¿es adecuado según lo especificado en la guía? SI o NO.', lines = 4)
              inp_pregunta3 = gr.Textbox(label="Pregunta 3", value='La descripción del cuadro clinico del paciente ¿es adecuada y completa según lo especificado en la guía? SI o NO.', lines = 4)
              inp_pregunta4 = gr.Textbox(label="Pregunta 4", value='¿Se le entregaron al paciente recomendaciones y signos de alarma de acuerdo con lo especificado en la guía? SI o NO.', lines = 4)
              inp_pregunta5 = gr.Textbox(label="Pregunta 5", value='¿Los paraclinicos solicitados fueron adecuados según lo especificado en la guía? SI o NO.', lines = 4)
      with gr.Accordion("RESULTADOS", open=True):
        outRespuestas = gr.Dataframe(label="Respuestas", visible= True, interactive= False)
      btn_procesar = gr.Button("PROCESAR EHR Y GPC")
      btn_procesar.click(cargar_procesar_configurar_historias_y_responder_preguntas,inputs=[inp_xls, inp_pdf, inp_pregunta1, inp_pregunta2, inp_pregunta3, inp_pregunta4, inp_pregunta5],outputs=outRespuestas)



    with gr.Tab("EVALUAR Y GRAFICAR"):
      gr.HTML("<p style='text-align:center'><b>AQUI SE VISUALIZA LA ADHERENCIA A LA GUIA SELECCIONADA</b></p>")
      with gr.Accordion("EVALUACION DE ADHERENCIA POR CRITERIOS", open=False):
          with gr.Row():
              input = outRespuestas ##outRespuestas
              output = gr.Dataframe(label="Respuestas", visible= True)
      btn_procesar = gr.Button("VISUALIZAR RESULTADOS")
      btn_procesar.click(minuscula,inputs=input,outputs=output)

      with gr.Accordion("MEDIR ADHERENCIA", open=False):
          with gr.Row():
              inputs = outRespuestas
              adherencia = gr.Dataframe(interactive=False)
      btn_procesar = gr.Button("MEDIR ADHERENCIA POR CRITERIOS")
      btn_procesar.click(limpiar_dataframe,inputs=inputs, outputs=adherencia)


      with gr.Accordion("GRAFICAR", open=False):
          with gr.Row():
              input = adherencia
              output = gr.Plot(show_label= True)
      btn_procesar = gr.Button("GRAFICAR ADHERENCIA GLOBAL Y POR CRITERIOS")
      btn_procesar.click(GRAFICO_BARRAS,inputs=input,outputs=output)

demo.launch(share=True, debug=True)

#TENER EN CUENTA QUE EL PROTOTIPO PUEDE REQUERIR AJUSTES DADO ACTUALIZACIONES EN LIBRERIAS NECESARIAS, SOBRETODO EN METODO APPEND DE PANDAS Y METODO RUN DE LANGCHAIN.
#VERSION ACTUAL DE CARACTER ACADEMICO, SE RECOMIENDA USAR VERSION CON PROVEEDOR PARA NUEVAS PRUEBAS

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4382238ce1fc2c2cf3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
!python -m pip freeze

absl-py==1.4.0
accelerate==0.25.0
aiofiles==23.2.1
aiohttp==3.9.3
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
antlr4-python3-runtime==4.9.3
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.5.0
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
backoff==2.2.1
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==0.23.0
bitsandbytes==0.42.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.1
build==1.1.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.85
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.1
colorama==0.4.6
colorcet==3.1.0
coloredlogs==15.0.1
colorlover==0.3.0
colour==0.1.5
communit